In [2]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.io import curdoc

def my_tap_handler(attr,old,new):
    index = source.selected.indices
    print(index)

source = ColumnDataSource(data=dict(
    x=[[1,2,3],[7,8,8,]],
    y=[[2,1,3],[6,8,7]]
))
p = figure(tools="tap")

renderer = p.patches('x', 'y', source=source)
renderer.data_source.on_change("selected", my_tap_handler)
curdoc().add_root(p)
show(p)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



In [8]:
import numpy as np
import pandas as pd
from bokeh.models.callbacks import CustomJS
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.layouts import row
from bokeh.models.widgets import DataTable, TableColumn
from bokeh.models import ColumnDataSource, CustomJSFilter, TapTool, HoverTool


source = ColumnDataSource(pd.DataFrame(
    np.array([[i, i] for i in range(0, 10)]), columns=['x', 'y']))

data = [[i, "%s.%d" % ("data", i)] for i in range(0, 10)]
data.extend([[i, "%s.%d" % ("data", i)] for i in range(0, 10)])
source2 = ColumnDataSource(pd.DataFrame(data, columns = ['x', 'data']))

columns = [
        TableColumn(field='x', title='x'),
        TableColumn(field='data', title='data'),
    ]
data_table = DataTable(source=source2, columns=columns, width=600, height=400)

selection_callback = CustomJS(args=dict(source=source, source2=source2, p2=data_table), code="""
    var indices = []

    var source_x = source.data.x[source.selected.indices[0]]
    source2.data.x.forEach((source2_x, i) => {
        if (source2_x == source_x) {
          indices.push(i)
        }
    });

    source2.selected.indices = indices
    p2.change.emit()
""")

plot = figure(title="Plot1", tools="tap,pan,box_zoom,wheel_zoom,save,reset")
plot.select(TapTool).callback = selection_callback
source.selected.js_on_change('indices', selection_callback)
plot.circle('x', 'y', source=source, line_color=None, color='red', size=6)

show(row(plot, data_table))

In [1]:
from random import random
from bokeh.layouts import row 
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.plotting import figure, show 

x = [random() for x in range(1000)]
y = [random() for y in range(1000)]

s1 = ColumnDataSource(data=dict(x=x, y=y))
p1 = figure(plot_width = 500, plot_height = 500, tools="lasso_select", title="Make Your Selection Here")
p1.circle('x', 'y', source=s1, alpha=0.7)

s2 = ColumnDataSource(data=dict(x=[], y=[]))
p2 = figure(plot_width=500, plot_height=500, x_range=(0,1), y_range=(0,1),
tools="save", title="Look Over Here")
p2.circle('x','y', source=s2, alpha=0.7)

## Callback
s1.selected.js_on_change('indices', CustomJS(args=dict(s1=s1,s2=s2),
code="""
var inds = cb_obj.indices;
var d1 = s1.data;
var d2 = s2.data;
d2['x'] = []
d2['y'] = []

for (var i = 0; i < inds.length; i++){
	d2['x'].push(d1['x'][inds[i]])
	d2['y'].push(d1['y'][inds[i]])
}

s2.change.emit(); // push data to python 
"""
))

# Layout
layout = row(p1, p2)
show(layout)


In [40]:
# Import Stuffs
from bokeh import events
from bokeh.plotting import figure, output_file, show, save
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.models import HoverTool, TapTool
from bokeh.models import ColumnDataSource, CustomJS, Div
from bokeh.events import Tap
from IPython.display import IFrame
from IPython.core.display import display, HTML
import tempfile
import numpy as np

In [3]:
# Deepnote Show function for Bokhe
def bokeh_deepnote_show(plot):
    tmp_output_filename = tempfile.NamedTemporaryFile(suffix='.html').name
    output_file(tmp_output_filename)
    save(plot)

    f = open(tmp_output_filename, "r")
    display(HTML(f.read()))

In [48]:
source = ColumnDataSource(dict(
    x=[1, 2, 2, 1, 2, 3],
    y=[1, 1, 2, 2, 3, 2],
    indicies=[0,1,2,3,4,5]
))

fig = figure(title='My Coordinates: Click Anywhere to Mark',
             plot_height=500, plot_width=600,
             x_range=(0, 4), y_range=(0, 4))

hover_glyph = fig.circle(source=source, x='x', y='y',
           color='green', size=10, alpha=0.5,
           hover_fill_color='black', hover_alpha=0.5)

tooltips = [('x', '@x'), ('y', '@y'), ('index', '$index'), ('indicies', '@indicies')]

fig.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))

callback = CustomJS(args=dict(source=source), code="""
    // change data source from Callback
    var mainData = source.data;
    var inds = mainData['indicies'];
    mainData['x'].push(cb_obj.x);
    mainData['y'].push(cb_obj.y);
    mainData['indicies'].push(inds[inds.length-1]+1);
    console.log(mainData);
    source.change.emit();
""")

fig.js_on_event(Tap, callback)

args=dict(source=source)
print(args)

bokeh_deepnote_show(fig)

{'source': ColumnDataSource(id='7989', ...)}


In [28]:
def display_event(div, attributes=[]):
    """
    Function to build a suitable CustomJS to display the current event
    in the div model.
    """
    style = 'float: left; clear: left; font-size: 13px'
    return CustomJS(args=dict(div=div), code="""
        const attrs = %s;
        const args = [];
        for (let i = 0; i < attrs.length; i++) {
            const val = JSON.stringify(cb_obj[attrs[i]], function(key, val) {
                return val.toFixed ? Number(val.toFixed(2)) : val;
            })
            args.push(attrs[i] + '=' + val)
        }
        const line = "<span style=%r><b>" + cb_obj.event_name + "</b>(" + args.join(", ") + ")</span>\\n";
        const text = div.text.concat(line);
        const lines = text.split("\\n")
        if (lines.length > 35)
            lines.shift();
        div.text = lines.join("\\n");
    """ % (attributes, style))

In [43]:
source = ColumnDataSource(dict(
    x=[1, 2, 2, 1, 2, 3],
    y=[1, 1, 2, 2, 3, 2],
))

selected = ColumnDataSource(dict(
    x=[],
    y=[],
))

fig = figure(title='My Coordinates: Select the Circles',
             plot_height=500, plot_width=600,
             x_range=(0, 4), y_range=(0, 4))

hover_glyph = fig.circle(source=source, x='x', y='y',
           color='green', size=10, alpha=0.5,
           hover_fill_color='black', hover_alpha=0.5)

tooltips = [('x', '@x'), ('y', '@y')]

fig.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))
fig.add_tools(TapTool())

current_indices

callback = CustomJS(args=dict(source=source, selected=selected, current_indices=current_indices), code="""
    // get data source from Callback args
    let data = Object.assign({}, source.data);
    let indices = Object.assign([], source.selected.indices);
    data.x = indices.map(i => data.x[i]);
    data.y = indices.map(i => data.y[i]);
    selected.data = data;
    current_indices = source.selected.indices
    console.log(current_indices);
    selected.change.emit();
""")

fig.js_on_event(Tap, callback)

bokeh_deepnote_show(fig)

print(current_indices)

0
